In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.applications.vgg16 import VGG16
import cv2
import math
import os
from glob import glob
from scipy import stats as s
from PIL import Image
from sklearn.metrics import accuracy_score


import PIL

from google.colab import files
uploaded = files.upload()

arr = os.listdir("/content/")
num = len(os.listdir("/content/sample_data/")) + len(os.listdir("/content/temp/"))
+ len(os.listdir("/content/test_1/")) + + len(os.listdir("/content/train_1/"))

numofvids = len(arr) - num - 1
f = open("trainlist01.txt","w+")
arr.sort()

arrofvids = []
for a in range (len(arr)):
  if(arr[a].endswith(".avi")):
    arrofvids.append(arr[a])

for i in range (0,len(arrofvids)//2):
  f.write(arrofvids[i] + "\n")
f.close()

f = open("testlist01.txt","w+")
for i in range (len(arrofvids)//2, len(arrofvids),1):
  f.write(arrofvids[i] + "\n")
f.close()
print("please read", len(arrofvids))

# open the .txt file which have names of training videos
f = open("trainlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
train = pd.DataFrame()
train['video_name'] = videos
train = train[:-1]
train.head()

train_video_tag = []
for i in range(train.shape[0]):
    index = train['video_name'][i].find("-")
    train_video_tag.append(train['video_name'][i][:index])
train['tag'] = train_video_tag


# open the .txt file which have names of training videos
f = open("testlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test.head()


# creating tags for test videos
test_video_tag = []
for i in range(test.shape[0]):
    test_video_tag.append(test['video_name'][i][:index])
test['tag'] = test_video_tag

# storing the frames from training videos
for i in tqdm(range(train.shape[0])):
    count = 0
    videoFile = train['video_name'][i]
    cap = cv2.VideoCapture('/content/'+videoFile)   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames in a new folder named train_1
            filename ='train_1/' + videoFile.split('-')[0]+"_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()

totallist = os.listdir("/content/train_1/")
print("this is total lenght of train", len(totallist))

# getting the names of all the images
images = glob("train_1/*.jpg")
train_image = []
train_class = []
for i in tqdm(range(len(images))):
    # creating the image name
    train_image.append(images[i].split('/')[1])
    # creating the class of image
    train_class.append(images[i].split('/')[1].split('_')[0])

# storing the images and their class in a dataframe
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

# converting the dataframe into csv file
train_data.to_csv('/content/train_new.csv',header=True, index=False)

In [ ]:

import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split


train = pd.read_csv('/content/train_new.csv')
train.head()


# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('train_1/'+train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)

# converting the list to numpy array
X = np.array(train_image)
#X = np.stack(train_image)

# shape of the array
X.shape

# separating the target
y = train['class']

print("type.  ", type(y))


print("this is y ", y)

# creating the training and validation set
print("length of train", len(X))
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5,stratify = y)
print("hi", len(X))
print(len(y))
X_train = X[:len(X)//2]
X_test = X[len(X)//2:]
y_train = y[:len(y)//2]
y_test = y[len(y)//2:]

# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

print(y_train.columns)
print(y_test.columns)
# import keras
# import keras.utils
# from keras import utils as utils
# from keras.utils import to_categorical
# import tensorflow as tf
# y_train = tf.keras.utils.to_categorical(y_train, 7)
# y_test = tf.keras.utils.to_categorical(y_test, 7)

print(y_train.shape)
print(y_test.shape)


# creating the base model of pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)


# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape


# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

print(X_test.shape)
print(X_train.shape)

# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(38, 7*7*512)
X_test = X_test.reshape(38, 7*7*512) #first num of shape


# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max



# shape of images
X_train.shape



#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
#model.add(Dense(101, activation='softmax'))
model.add(Dense(24, activation='softmax')) #last num of shape

# tf.keras.utils.to_categorical(
#     y_train, num_classes=None, dtype='float32'
# )


In [ ]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('imagenet', save_best_only=True, monitor='val_loss', mode='min')

# compiling the model
#model.compile(loss='sparse_categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()
#model.add( keras.layers.Flatten())

import keras
import keras.utils
from keras import utils as utils
from keras.utils import to_categorical
import tensorflow as tf


# y_train = tf.reshape(y_train, (-1, 1))
# y_test = tf.reshape(y_test, (-1, 1))
# y_train = tf.reshape(y_train, [7,4])
# y_test = tf.reshape(y_test, [2,2])


# print(y_train.shape)
# print(y_test.shape)

# y_train = to_categorical(y_train, 4)
# y_test = to_categorical(y_test, 4)

#to_categorical(y_train, num_classes=None, dtype='float32')
print("y_train", y_train.shape)
print("y_test", y_test.shape)
print("X_train", X_train.shape)
print("X_test", X_test.shape)



# training the model
#model.fit(X_train, y_train, batch_size=128, epochs=200, validation_data=(X_test, y_test))
#model.fit(X_train, y_train, batch_size=128, epochs=2, validation_data=(X_test, y_test))
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.applications.vgg16 import VGG16
import cv2
import math
import os
from glob import glob
from scipy import stats as s

base_model = VGG16(weights='imagenet', include_top=False)

#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(24, activation='softmax')) #last num


# loading the trained weights
#model.load_weights("weights.hdf5")

# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

# getting the test list
f = open("testlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating the dataframe
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test_videos = test['video_name']
test.head()

# creating the tags
train = pd.read_csv('/content/train_new.csv')
y = train['class']

y = pd.get_dummies(y)

# creating two lists to store predicted and actual tags
predict = []
actual = []
# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    cap = cv2.VideoCapture('/content/'+videoFile)   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    # removing all other files from the temp folder
    files = glob('temp/*')
    for f in files:
        os.remove(f)
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames of this particular video in temp folder
            filename ='temp/' + "_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()

    # reading all the frames from temp folder
    images = glob("temp/*.jpg")

    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255
        prediction_images.append(img)

    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    # extracting features using pre-trained model
    prediction_images = base_model.predict(prediction_images)
    # converting features in one dimensional array
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*512)
    # predicting tags for each array
    prediction = model.predict_classes(prediction_images)
    # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(y.columns.values[s.mode(prediction)[0][0]])
    # appending the actual tag of the video
    actual.append(videoFile.split("-")[0])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100